In [1]:
## Section 1: Importing Libraries
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator

## Section 2: Dataset Preparation
# Paths to the dataset
TRAIN_DIR = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Dataset\DATA"
TEST_DIR = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Dataset\TEST"
LABELS_PATH = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\labels.csv"

# Load labels
labels_df = pd.read_csv(LABELS_PATH)
class_names = labels_df.set_index('ClassId')['Name'].to_dict()

def load_train_data(train_dir):
    images, labels = [], []
    class_folders = os.listdir(train_dir)
    for class_id in class_folders:
        class_path = os.path.join(train_dir, class_id)
        for img_name in os.listdir(class_path):
            img_path = os.path.join(class_path, img_name)
            img = cv2.imread(img_path)
            img = cv2.resize(img, (32, 32))  # Resize to 32x32
            images.append(img)
            labels.append(int(class_id))
    return np.array(images), np.array(labels)

def load_test_data(test_dir):
    images = []
    for img_name in os.listdir(test_dir):
        img_path = os.path.join(test_dir, img_name)
        img = cv2.imread(img_path)
        img = cv2.resize(img, (32, 32))  # Resize to 32x32
        images.append(img)
    return np.array(images)

# Load datasets
X_train, y_train = load_train_data(TRAIN_DIR)
X_test = load_test_data(TEST_DIR)

# Normalize images
X_train = X_train / 255.0
X_test = X_test / 255.0

# One-hot encode labels
y_train = to_categorical(y_train)

## Section 3: Model Creation
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(len(labels_df), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

## Section 4: Model Training
BATCH_SIZE = 32
EPOCHS = 10

# Data Augmentation
data_gen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.1
)
data_gen.fit(X_train)

history = model.fit(
    data_gen.flow(X_train, y_train, batch_size=BATCH_SIZE),
    epochs=EPOCHS
)
## Section 5: Model Evaluation

# Note: Since the test data does not have labels, accuracy cannot be directly evaluated on the test set.
# We will use the training set's split for validation during training instead.

loss, accuracy = model.evaluate(X_train, y_train)
print(f"Training Accuracy: {accuracy * 100:.2f}%")

## Section 6: Traffic Sign Detection

# Load the saved model (if already trained)
model.save('traffic_sign_model.h5')
model = load_model('traffic_sign_model.h5')

def predict_sign(image_path):
    """Predict the traffic sign from an input image."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (32, 32)) / 255.0
    img_expanded = np.expand_dims(img_resized, axis=0)
    predictions = model.predict(img_expanded)
    class_id = np.argmax(predictions)
    confidence = np.max(predictions)
    return class_names[class_id], confidence

# Example usage for single image input
image_path = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Dataset\TEST\011_0019_j.png"
sign_name, conf = predict_sign(image_path)
print(f"Predicted Sign: {sign_name}, Confidence: {conf:.2f}")


## Example usage for prediction
import matplotlib.pyplot as plt
import cv2
import numpy as np

def predict_sign(image_path):
    """Predict the traffic sign from an input image."""
    img = cv2.imread(image_path)
    img_resized = cv2.resize(img, (32, 32)) / 255.0
    img_expanded = np.expand_dims(img_resized, axis=0)
    predictions = model.predict(img_expanded)
    class_id = np.argmax(predictions)
    confidence = np.max(predictions)
    return class_names[class_id], confidence, img  # Return the image for display

# Example usage for single image input
image_path = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Dataset\TEST\011_0019_j.png"
sign_name, conf, img = predict_sign(image_path)

# Display the image and the prediction result
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convert BGR to RGB for correct color display
plt.axis('off')  # Hide axes
plt.title(f"Predicted Sign: {sign_name}, Confidence: {conf:.2f}")
plt.show()

print(f"Predicted Sign: {sign_name}, Confidence: {conf:.2f}")

## Example_2_usage for detection
def detect_traffic_signs(image_path):
    """Detect and classify traffic signs in an image."""
    # Load and preprocess the image
    image = cv2.imread(image_path)
    original_image = image.copy()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Detect edges and contours (basic traffic sign detection logic)
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)
    edges = cv2.Canny(blurred, 50, 150)
    contours, _ = cv2.findContours(edges, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

    best_confidence = 0
    best_box = None
    best_label = None

    for contour in contours:
        # Approximate the contour to filter out small or irregular shapes
        approx = cv2.approxPolyDP(contour, 0.02 * cv2.arcLength(contour, True), True)
        x, y, w, h = cv2.boundingRect(approx)

        if w > 20 and h > 20:  # Filter small boxes
            # Extract and preprocess the region of interest (ROI)
            roi = image[y:y+h, x:x+w]
            roi_resized = cv2.resize(roi, (32, 32)) / 255.0
            roi_expanded = np.expand_dims(roi_resized, axis=0)

            # Predict the class of the ROI
            predictions = model.predict(roi_expanded)
            class_id = np.argmax(predictions)
            confidence = np.max(predictions)

            # Keep track of the best detection
            if confidence > best_confidence:
                best_confidence = confidence
                best_box = (x, y, w, h)
                best_label = f"{class_names[class_id]} ({confidence:.2f})"

    # Draw the best bounding box and label if found
    if best_box and best_confidence > 0.5:
        x, y, w, h = best_box
        cv2.rectangle(original_image, (x, y), (x + w, y + h), (0, 0, 255), 2)  # Red box
        cv2.putText(original_image, best_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)

    # Show the result inline (e.g., in Jupyter Notebook)
    plt.figure(figsize=(10, 10))
    plt.imshow(cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB))
    plt.axis('off')
    plt.title('Detected Traffic Signs')
    plt.show()

# Example usage for detecting traffic signs in an image
image_path = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Dataset\TEST\030_0017_j.png"
detect_traffic_signs(image_path)

## Section 7: Traffic Sign Detection in Video
def process_video(video_path, output_path):
    """Detect traffic signs in a video and save output."""
    cap = cv2.VideoCapture(video_path)
    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(output_path, fourcc, 20.0, (int(cap.get(3)), int(cap.get(4))))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        frame_resized = cv2.resize(frame, (32, 32)) / 255.0
        frame_expanded = np.expand_dims(frame_resized, axis=0)
        predictions = model.predict(frame_expanded)
        class_id = np.argmax(predictions)
        confidence = np.max(predictions)

        if confidence > 0.5:  # Threshold confidence
            label = f"{class_names[class_id]} ({confidence:.2f})"
            cv2.putText(frame, label, (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.rectangle(frame, (10, 10), (300, 300), (0, 0, 255), 2)

        out.write(frame)
        cv2.imshow('Traffic Sign Detection', frame)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    out.release()
    cv2.destroyAllWindows()

# Example usage for video processing
video_path = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Traffic sign video.mp4"
output_path = r"C:\ITC Year 4\I4 Artificial Intelligent (TP)\Final Project AI ( TP )\Traffic sign video.avi"
process_video(video_path, output_path)





KeyboardInterrupt: 